In [227]:
import os
import tensorflow as tf
import io
import numpy as np
import tensorflow_addons as tfa

In [228]:
#Define variables

batch_size = 32
img_height = 256
img_width = 256
img_channels = 3

In [229]:
#Normalize pixel function

@tf.autograph.experimental.do_not_convert
def _normalize_img(img, label):
    image = tf.cast(img/255. ,tf.float32)
    return image, label

In [230]:
#Update Data Directory Name

data_dir = 'data_folder/all_data_processed' #Change this to lfw & pubfig dataset

In [231]:
#Load Dataset

lfw_pubfig_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  shuffle=True,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

lfw_pubfig_ds_norm = lfw_pubfig_ds.map(_normalize_img)

Found 20804 files belonging to 1751 classes.


In [232]:
#Set Dataset Split 0.7/0.15/0.15

train_split = 0.7
val_split = 0.15
test_split = 0.15

In [233]:
#Split Dataset

ds_size = len(list(lfw_pubfig_ds_norm))
train_size = int(train_split * ds_size)
val_size = int(val_split * ds_size)
test_size = int(test_split * ds_size)

train_ds = lfw_pubfig_ds_norm.take(train_size)    
val_ds = lfw_pubfig_ds_norm.skip(train_size).take(val_size)
test_ds = lfw_pubfig_ds_norm.skip(train_size + val_size).take(test_size)

In [234]:
#Print Dataset Sizes

print(len(list(train_ds)) * batch_size, train_ds.element_spec) # Default Batches of 32
print(len(list(val_ds)) * batch_size, val_ds.element_spec) # Default Batches of 32
print(len(list(test_ds)) * batch_size, test_ds.element_spec) # Default Batches of 32

14560 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
3104 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
3104 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [9]:
#Create the Baseline Model Class

class BaselineModel(tf.keras.Model):
    def __init__(self):
        super(BaselineModel, self).__init__()
        
        self.input_layer = tf.keras.layers.Input((img_height, img_width, img_channels))
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=7, padding='same', activation='relu')
        self.mpool1 = tf.keras.layers.MaxPooling2D(pool_size=2)
        
        self.flat1 = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, activation=None)

        self.output_layer = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.mpool1(x)
        x = self.flat1(x)
        x = self.fc1(x)
        out = self.output_layer(x)
        
        return out
    
    def model(self):
        return tf.keras.Model(inputs=self.input_layer, outputs=self.call(self.input_layer))
        

In [45]:
#Create the Advanced Model Class (Insert Model Architecture)

class AdvancedModel(tf.keras.Model):
    def __init__(self):
        super(AdvancedModel, self).__init__()
        
        self.input_layer = tf.keras.layers.Input((img_height, img_width, img_channels))
        self.c1 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=(2, 2), padding='same', activation='relu')
        self.mp1 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='valid', strides=(2, 2))
        self.d1 = tf.keras.layers.Dropout(0.3)
        
        self.c2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=(2, 2), padding='same', activation='relu')
        self.mp2 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='valid', strides=(2, 2))
        self.d2 = tf.keras.layers.Dropout(0.3)
        
        self.c3 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(2, 2), padding='same', activation='relu')
        self.mp3 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='valid', strides=(2, 2))
        
        self.flat1 = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, activation=None)

        self.output_layer = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        
    def call(self, inputs):
        x = self.c1(inputs)
        x = self.mp1(x)
        x = self.d1(x)
        
        x = self.c2(x)
        x = self.mp2(x)
        x = self.d2(x)
        
        x = self.c3(x)
        x = self.mp3(x)
        
        x = self.flat1(x)
        x = self.fc1(x)
        
        out = self.output_layer(x)
        
        return out
    
    def model(self):
        return tf.keras.Model(inputs=self.input_layer, outputs=self.call(self.input_layer))

In [241]:
#Get Model

#Uncomment for Baseline Model
model_name = 'baseline'
model = BaselineModel()

#Uncomment for Advanced Model
#model_name = 'advanced'
#model = AdvancedModel()

#Uncomment to load a Model
#model = tf.keras.models.load_model('saved_model/' + model_name + '_model')

In [239]:
#Compliel Model with Adam Optimizer and Triplet Loss

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tfa.losses.TripletHardLoss())

In [240]:
#Train Model
EPOCHS = 50

history = model.fit(
        x=train_ds,
        epochs=EPOCHS,
        validation_data=val_ds)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


KeyboardInterrupt: 

In [49]:
#Save Model
model.save('saved_model/' + model_name + '_model')

INFO:tensorflow:Assets written to: saved_model/advanced_model\assets


INFO:tensorflow:Assets written to: saved_model/advanced_model\assets


In [50]:
#Save training data
import pandas as pd

# convert the history.history dict to a pandas DataFrame:   
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'saved_model/' + model_name + '_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [248]:
#Constuct a database of known image encodings from the training/verification dataset as a reference to evaluate the test dataset

pubfig_db_dict = {}
for images, labels in train_ds.take(len(list(train_ds))):
    for i in range(labels.shape[0]):
        if int(labels[i]) not in pubfig_db_dict:
            pubfig_db_dict[int(labels[i])] = model.predict(np.expand_dims(images[i], axis=0))
            
for images, labels in val_ds.take(len(list(val_ds))):
    for i in range(labels.shape[0]):
        if int(labels[i]) not in pubfig_db_dict:
            pubfig_db_dict[int(labels[i])] = model.predict(np.expand_dims(images[i], axis=0))

1/1 [==============================] - 0s 98ms/step


1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 99ms/step


1/1 [==============================] - 0s 74ms/step


1/1 [==============================] - 0s 81ms/step


...

1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 81ms/step


1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 73ms/step


In [245]:
#Create lists from reference dict
label_list = np.squeeze(np.array(list(pubfig_db_dict.keys())))
ref_list = np.squeeze(np.array(list(pubfig_db_dict.values())))
print(ref_list.shape)

(1729, 128)


In [246]:
#Make Model Predictions (vectorized by batch)
test_num = 0
test_correct = 0

threshold = 1.3
test_samples = len(list(test_ds))

for images, labels in test_ds.take(test_samples):
    #predict batch
    batch_pred = model.predict(images)
    
    #Calculate norm differerecen for each embedding and all references
    norm_diffs = np.zeros((ref_list.shape[0], batch_pred.shape[0]))
    for i, ref in enumerate(ref_list):
        norm_diffs[i] = np.linalg.norm(batch_pred - ref, axis=1)
    norm_diffs = norm_diffs.transpose()
    
    #Get the min labels and test labels
    min_labels = np.argmin(norm_diffs, axis=1)
    
    test_num += len(min_labels)
    for i in range(len(min_labels)):
        min_index = min_labels[i]
        test_label = labels[i]
        label = label_list[min_index]
    
        if norm_diffs[i][min_index] < threshold:
            if label == test_label:
                test_correct += 1
        elif test_label not in pubfig_db_dict:
            test_correct += 1

1/1 [==============================] - 1s 864ms/step


In [247]:
#Print Facial Recognition Accuracy and Average Distance from correct class

print(test_correct/test_num)
print(total_distance/test_num)

0.053479381443298966
0.0
